In [13]:
# # https://catboost.ai/en/docs/concepts/python-reference_catboost
# import catboost
# pooling = catboost.Pool()
# model = catboost.CatBoost()
# model.fit(train_pool)
# model.save_model()

# def apply_catboost_model(float_features, cat_features):
#     augment 2 + a, b, c -> 예상 스코어가 높은걸로 하면 됨.
#     애초에 시너지로 이걸 바꾼순간 w -> 0
#     prediction ->

# 증강체 등급 구별

from catboost import CatBoostRegressor, Pool
import csv
import re
import json
import ipywidgets

f = open("C:/Users/edwar/TFTML/data/sorted_temp2.csv", "r")
# f = open('./data/sorted_temp2.csv', 'r') len:2555
# data len: 2555*8 = 20440
rdr = csv.DictReader(f)
## Feature list import
MetaData_item = []
with open("C:/Users/edwar/TFTML/data/metaList2.txt", "r") as a:
    s = a.readlines()
for line in s:
    res = re.sub("'|,| |\n", "", line)
    MetaData_item.append(res)


X = []
valid = []
label = []

for i in rdr:
    _dict = json.loads(i["info.participants"].replace("'", '"'))
    for j in _dict:
        X_i = []
        # traits = []
        # items = []
        X_i.extend(j["augments"])
        for k in j["traits"]:
            if k["name"] + str(k["style"]) in MetaData_item:
                X_i.append(k["name"] + str(k["style"]))
        for h in j["units"]:
            if h["itemNames"] != []:
                item_list = list(h["itemNames"])
                if "TFT_Item_ThiefsGloves" in item_list:
                    X_i.append("TFT_Item_ThiefsGloves")
                else:
                    for g in range(len(h["itemNames"]), -1):
                        # if (item_list[g] not in MetaData_item) or (item_list[g] in del_metaList):
                        #     del item_list[g]
                        if item_list[g] in MetaData_item:
                            X_i.append(item_list[g])

        label.append(j["placement"])
        X.append(sorted(X_i))  # sorted

X = [[1 if i in row_x else 0 for i in MetaData_item] for row_x in X]
# multi-hot_encoding
len = len(label)
train_X = X[0 : len // 2]
train_label = label[0 : len // 2]
valid_X = X[len // 2 : len * 3 // 4]
valid_label = label[len // 2 : len * 3 // 4]
test_X = X[len * 3 // 4 : len]
test_label = label[len * 3 // 4 : len]
train_data = Pool(data=train_X, label=train_label)
valid_data = Pool(data=valid_X, label=valid_label)

print(train_data)
"""
catboost가 pandasFrame이 필요해보임
column이 358개 각 feature들어가고, 
각 게임 데이터 row마다 1 or 0 ㄱ backup2쓰면 될듯
"""

model = CatBoostRegressor(
    iterations=5000,
    task_type="GPU",
)
# x_train, x_val = X[:int(len(X)/2)], X[int(len(X)/2):]
# y_train, y_val = label[:int(len(X)/2)], label[int(len(X)/2):]
# model.fit(x_train, y_train,cat_features=list(range(len(MetaData_item))),
#           eval_set=(x_val, y_val))
model.fit(train_data, eval_set=valid_data, plot=True)
print(model.get_best_iteration())
print(model.get_best_score())
preds_class = model.predict(X)
# print(f"train_data:\n{train_data}\n")
print(preds_class[:10])


TypeError: 'int' object is not callable